# **Import Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
import seaborn as sns
import cv2 as cv
import csv
from scipy import ndimage # permite utilizar uma função semelhante ao imfilter do matlab

from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn import ensemble
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.utils import to_categorical

%matplotlib inline

# **Feature Extraction**

In [2]:
#Primeira parte do dataset-> 20000 imagens fake 
path= "dataset_train/dataset_train/fake/image_{0:d}.png"
data=np.arange(785).ravel()
for i in range (20000):
    file_path=path.format(i)
    img=cv.imread(file_path)
    gray_image =cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    Class0 = np.array([gray_image])
    Class0=np.ravel(Class0)
    data=np.vstack([data, np.insert(Class0,0,0)]) 

print("treino fake -> 50% done")
#Segunda parte do dataset->20000 imagens real
path= "dataset_train/dataset_train/real/image_{0:d}.png"
for j in range(20000):
    file_path=path.format(j)
    img=cv.imread(file_path)
    gray_image=cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    Class1= np.array(gray_image)
    Class1=np.ravel(Class1)
    data=np.vstack([data, np.insert(Class1,0,1)])

data=np.delete(data,0,0)
print("treino real -> 100% done")
print(data.shape)

result=['label']

for i in range(1,785):
    result.append('pixel'+str(i))

print(len(result))

info=pd.DataFrame(data, columns=result)

info.to_csv('dataset_treino.csv', index=False)
print(info) 

treino fake -> 50% done
treino real -> 100% done
(40000, 785)
785
       label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0          0       0       0       0       0       0       0       0       0   
1          0       0       0       0       0       0       0       0       0   
2          0       0       0       0       0       0       0       0       0   
3          0       0       0       0       0       0       0       0       6   
4          0       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
39995      1       0       0       0       0       0       0       0       0   
39996      1       0       0       0       0       0       0       0       0   
39997      1       0       0       0       0       0       0       0       0   
39998      1       0       0       0       0       0       0       0       0   
39999      1       0       0       0       0       0  

In [3]:
path= "dataset_test\dataset_test\images/{0:d}.png"
data=np.arange(784).ravel()
for i in range (0,2000):   
    file_path=path.format(i)
    img=cv.imread(file_path)
    gray_img =cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    teste_img = np.array(gray_img)
    teste_img = np.ravel(teste_img)
    data=np.vstack([data, teste_img]) 
    
data=np.delete(data,0,0)

result=[]
for i in range(1,785):
    result.append('pixel'+str(i))
    
info=pd.DataFrame(data)

info.to_csv('dataset_teste.csv', index=False)
print("teste->100%") 
data.shape

teste->100%


(2000, 784)

# **Feature extractiom e sobel filter**

In [4]:
def sobel(img):
    
    kernelx=np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    kernely=np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    sobelx=ndimage.filters.convolve(img,kernelx)
    sobely=ndimage.filters.convolve(img,kernely)
    
    #sobelx=cv.Sobel(gray_image,cv.CV_64F,1,0,ksize=3)
    #sobely=cv.Sobel(gray_image,cv.CV_64F,0,1,ksize=3)
    
    modulo= np.sqrt(np.square(sobelx)+np.square(sobely))
    gradiente=np.arctan2(sobely,sobelx)
    teta=np.rad2deg(gradiente)
    
    m=n=28
    z=np.zeros((m,n),dtype=np.int32)

    for i in range(2,m-1):
        for j in range(2,n-1):
            z[i,j]=modulo[i,j]
            
            if( (teta[i,j]>-22.5 and teta[i,j]<=22.5) or (teta[i,j]>157.5 and teta[i,j]<=180) or (teta[i,j]<-157.5 and teta[i,j]>=-180)):
                if((modulo[i,j]<modulo[i,j-1]) or (modulo[i,j]<modulo[i,j+1])):
                    z[i,j]=0
            
            elif((teta[i,j]>22.5 and teta[i,j]<=67.5) or (teta[i,j]>=-157.5 and teta[i,j]<-112.5)):
                if((modulo[i,j]<modulo[i-1,j+1]) or (modulo[i,j]<modulo[i+1,j-1])):
                     z[i,j]=0
            
            elif((teta[i,j]>67.5 and teta[i,j]<=112.5) or (teta[i,j]>=-112.5 and teta[i,j]<-67.5)):
                if((modulo[i,j]<modulo[i-1,j]) or (modulo[i,j]<modulo[i+1,j])):
                    z[i,j]=0
            
            elif( (teta[i,j]>112.5 and teta[i,j]<=157.5) or (teta[i,j]>=-67.5 and teta[i,j]<-22.5)):
                if((modulo[i,j]<modulo[i-1,j-1]) or (modulo[i,j]<modulo[i+1,j+1])):
                    z[i,j]=0;      
    return z

#Primeira parte do dataset-> 20000 imagens fake 
path= "dataset_train/dataset_train/fake/image_{0:d}.png"
data=np.arange(785).ravel()
for i in range (20000):
    file_path=path.format(i)
    img=cv.imread(file_path)
    gray_image =cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    #aplicação do filtro de sobel
    image=sobel(gray_image)
    
    Class0 = np.array([image])
    Class0=np.ravel(Class0)
    data=np.vstack([data, np.insert(Class0,0,0)]) 

print("treino fake -> 50% done")
#Segunda parte do dataset->20000 imagens real
path= "dataset_train/dataset_train/real/image_{0:d}.png"
for j in range(20000):
    file_path=path.format(j)
    img=cv.imread(file_path)
    gray_image=cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    #aplicação do filtro de sobel
    image=sobel(gray_image)
    
    Class1= np.array(image)
    Class1=np.ravel(Class1)
    data=np.vstack([data, np.insert(Class1,0,1)])

data=np.delete(data,0,0)
print("treino real -> 100% done")
print(data.shape)

result=['label']

for i in range(1,785):
    result.append('pixel'+str(i))

print(len(result))

info=pd.DataFrame(data, columns=result)

info.to_csv('sobel_train.csv', index=False)
print(info) 

C:\Users\Utilizador\AppData\Local\Temp\ipykernel_6820\2492999795.py:6: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  sobelx=ndimage.filters.convolve(img,kernelx)
C:\Users\Utilizador\AppData\Local\Temp\ipykernel_6820\2492999795.py:7: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  sobely=ndimage.filters.convolve(img,kernely)


treino fake -> 50% done
treino real -> 100% done
(40000, 785)
785
       label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0          0       0       0       0       0       0       0       0       0   
1          0       0       0       0       0       0       0       0       0   
2          0       0       0       0       0       0       0       0       0   
3          0       0       0       0       0       0       0       0       0   
4          0       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
39995      1       0       0       0       0       0       0       0       0   
39996      1       0       0       0       0       0       0       0       0   
39997      1       0       0       0       0       0       0       0       0   
39998      1       0       0       0       0       0       0       0       0   
39999      1       0       0       0       0       0  

In [5]:
def sobel(img):
    
    kernelx=np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    kernely=np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    sobelx=ndimage.filters.convolve(img,kernelx)
    sobely=ndimage.filters.convolve(img,kernely)
    
    #sobelx=cv.Sobel(gray_image,cv.CV_64F,1,0,ksize=3)
    #sobely=cv.Sobel(gray_image,cv.CV_64F,0,1,ksize=3)
    
    modulo= np.sqrt(np.square(sobelx)+np.square(sobely))
    gradiente=np.arctan2(sobely,sobelx)
    teta=np.rad2deg(gradiente)
    
    m=n=28
    z=np.zeros((m,n),dtype=np.int32)

    for i in range(2,m-1):
        for j in range(2,n-1):
            z[i,j]=modulo[i,j]
            
            if( (teta[i,j]>-22.5 and teta[i,j]<=22.5) or (teta[i,j]>157.5 and teta[i,j]<=180) or (teta[i,j]<-157.5 and teta[i,j]>=-180)):
                if((modulo[i,j]<modulo[i,j-1]) or (modulo[i,j]<modulo[i,j+1])):
                    z[i,j]=0
            
            elif((teta[i,j]>22.5 and teta[i,j]<=67.5) or (teta[i,j]>=-157.5 and teta[i,j]<-112.5)):
                if((modulo[i,j]<modulo[i-1,j+1]) or (modulo[i,j]<modulo[i+1,j-1])):
                     z[i,j]=0
            
            elif((teta[i,j]>67.5 and teta[i,j]<=112.5) or (teta[i,j]>=-112.5 and teta[i,j]<-67.5)):
                if((modulo[i,j]<modulo[i-1,j]) or (modulo[i,j]<modulo[i+1,j])):
                    z[i,j]=0
            
            elif( (teta[i,j]>112.5 and teta[i,j]<=157.5) or (teta[i,j]>=-67.5 and teta[i,j]<-22.5)):
                if((modulo[i,j]<modulo[i-1,j-1]) or (modulo[i,j]<modulo[i+1,j+1])):
                    z[i,j]=0;      
    return z

path= "dataset_test\dataset_test\images/{0:d}.png"
data=np.arange(784).ravel()
for i in range (0,2000):   
    file_path=path.format(i)
    img=cv.imread(file_path)
    gray_img =cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    #aplicação do filtro de sobel
    image=sobel(gray_img)
    
    teste_img = np.array(image)
    teste_img = np.ravel(teste_img)
    data=np.vstack([data, teste_img]) 
    
data=np.delete(data,0,0)

result=[]
for i in range(1,785):
    result.append('pixel'+str(i))
    
info=pd.DataFrame(data)

info.to_csv('sobel_test.csv', index=False)
print("teste->100%") 
data.shape 

C:\Users\Utilizador\AppData\Local\Temp\ipykernel_6820\3379311660.py:6: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  sobelx=ndimage.filters.convolve(img,kernelx)
C:\Users\Utilizador\AppData\Local\Temp\ipykernel_6820\3379311660.py:7: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  sobely=ndimage.filters.convolve(img,kernely)


teste->100%


(2000, 784)

# **Data preparation**

In [2]:
data_train = pd.read_csv('dataset_treino.csv')
data_teste = pd.read_csv('dataset_teste.csv')


#data_train = pd.read_csv('sobel_train.csv')
#data_teste = pd.read_csv('sobel_test.csv')

labels_train = data_train.iloc[:,0]
features_train = data_train.iloc[:,1:]

labels_train = labels_train.to_numpy()
features_train = features_train.to_numpy()
features_test = data_teste.to_numpy()

features_train = features_train/255
features_test = features_test/255

In [3]:
(X_train, X_test, Y_train, Y_test) = train_test_split( features_train, labels_train ,test_size= 0.30,shuffle=True)

# **Ensembles**

## **Random Forest**

In [4]:
rnd = RandomForestClassifier(criterion='entropy', max_depth=15, min_samples_leaf=5,verbose=True).fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   30.1s finished


In [5]:
y_perd=rnd.predict(X_test)
accuracy_score(Y_test, y_perd)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


0.9214166666666667

In [6]:
teste_predicted=rnd.predict(features_test)
teste_predicted.sum()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


868

In [7]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_RForest.csv', index=False)

In [8]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.87      0.98      0.93      5964
           1       0.98      0.86      0.92      6036

    accuracy                           0.92     12000
   macro avg       0.93      0.92      0.92     12000
weighted avg       0.93      0.92      0.92     12000



## **Bagging**

In [9]:
bg = BaggingClassifier(warm_start = True, base_estimator=SVC(), n_estimators=2, random_state=0,verbose=True).fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min finished


In [10]:
y_perd=bg.predict(X_test)
accuracy_score(Y_test, y_perd)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.0min finished


0.9668333333333333

In [11]:
teste_predicted=bg.predict(features_test)
teste_predicted.sum()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.4s finished


997

In [12]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_Bagging.csv', index=False)

In [13]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      5964
           1       0.98      0.96      0.97      6036

    accuracy                           0.97     12000
   macro avg       0.97      0.97      0.97     12000
weighted avg       0.97      0.97      0.97     12000



## **Boosting**

In [14]:
bs = ensemble.GradientBoostingClassifier(n_estimators=400, learning_rate=0.1, max_depth=4, random_state=0,verbose=1).fit(X_train,Y_train)

      Iter       Train Loss   Remaining Time 
         1           1.3584           24.53m
         2           1.3377           23.99m
         3           1.3170           23.53m
         4           1.2944           23.30m
         5           1.2780           23.30m
         6           1.2617           23.21m
         7           1.2466           23.17m
         8           1.2347           23.11m
         9           1.2208           23.22m
        10           1.2064           23.47m
        20           1.0791           22.59m
        30           0.9830           21.94m
        40           0.9066           21.29m
        50           0.8454           20.75m
        60           0.7930           20.12m
        70           0.7418           19.51m
        80           0.7049           19.00m
        90           0.6590           18.33m
       100           0.6192           17.71m
       200           0.3598           11.61m
       300           0.2245            5.77m
       40

In [15]:
y_perd=bs.predict(X_test)
accuracy_score(Y_test, y_perd)

0.986

In [16]:
teste_predicted=bs.predict(features_test)
teste_predicted.sum()

986

In [17]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_Boosting.csv', index=False)

In [18]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5964
           1       0.99      0.98      0.99      6036

    accuracy                           0.99     12000
   macro avg       0.99      0.99      0.99     12000
weighted avg       0.99      0.99      0.99     12000



## **Majority Voting**

In [19]:
#acho que n é esta função(classificador) nas cenas do sklearn estimators é um array
clf1 = LogisticRegression(multi_class='multinomial', max_iter=5000, random_state=1,verbose=True)
clf2 = RandomForestClassifier(n_estimators = 500, random_state=1,verbose=True)
clf3 = MLPClassifier(hidden_layer_sizes=(250,40), max_iter=300,solver="adam",learning_rate_init=0.0001,verbose=True)

mv = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('mlp', clf3)], voting = 'hard',verbose=True,n_jobs=-1).fit(X_train,Y_train)
#mv = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft', flatten_transform=True).fit(X_train,Y_train)

In [20]:
y_perd = mv.predict(X_test)
accuracy_score(Y_test, y_perd)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    2.3s finished


0.9819166666666667

In [21]:
teste_predicted = mv.predict(features_test)
teste_predicted.sum()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.2s finished


1004

In [22]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_MVoting.csv', index=False)

In [23]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5964
           1       0.98      0.98      0.98      6036

    accuracy                           0.98     12000
   macro avg       0.98      0.98      0.98     12000
weighted avg       0.98      0.98      0.98     12000



## **Extra Trees**

In [24]:
et = ExtraTreesClassifier(n_estimators=300).fit(X_train,Y_train)

In [25]:
y_perd=et.predict(X_test)
accuracy_score(Y_test, y_perd)

0.9808333333333333

In [26]:
teste_predicted=et.predict(features_test)
teste_predicted.sum()

1010

In [27]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_ETrees.csv', index=False)

In [28]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5964
           1       0.98      0.98      0.98      6036

    accuracy                           0.98     12000
   macro avg       0.98      0.98      0.98     12000
weighted avg       0.98      0.98      0.98     12000



# **Decision Tree**

In [29]:
dt = DecisionTreeClassifier(criterion = 'entropy').fit(X_train,Y_train)

In [30]:
Y_train.sum()

13964

In [31]:
y_perd = dt.predict(X_test)
accuracy_score(Y_test, y_perd)

0.8785833333333334

In [32]:
teste_predicted=dt.predict(features_test)
d=np.arange(0, 2000, 1)
data=pd.DataFrame({'Id':d,'Category':teste_predicted})
data.to_csv('predict_Dtree.csv', index=False)

In [33]:
teste_predicted.sum()

959

In [34]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88      5964
           1       0.89      0.87      0.88      6036

    accuracy                           0.88     12000
   macro avg       0.88      0.88      0.88     12000
weighted avg       0.88      0.88      0.88     12000



# **K Means**

In [35]:
kmeans = KMeans(n_clusters=2, random_state = 0).fit(X_train, Y_train)

In [36]:
y_perd=kmeans.predict(X_test)
accuracy_score(Y_test, y_perd)

0.48191666666666666

In [37]:
teste_predicted=kmeans.predict(features_test)
teste_predicted.sum()

1160

In [38]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_KMeans.csv', index=False)

In [39]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.47      0.39      0.43      5964
           1       0.49      0.57      0.52      6036

    accuracy                           0.48     12000
   macro avg       0.48      0.48      0.48     12000
weighted avg       0.48      0.48      0.48     12000



# **KNN**

In [40]:
knn = KNeighborsClassifier(n_neighbors=3,weights='uniform',algorithm= 'ball_tree').fit(X_train,Y_train)

In [41]:
y_pred = knn.predict(X_test)
accuracy_score(Y_test, y_pred)

0.8848333333333334

In [42]:
teste_predicted = knn.predict(features_test)
teste_predicted = teste_predicted.reshape(2000,1)
d = np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data = np.hstack([d, teste_predicted])
data = pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_KNN.csv', index=False)

In [43]:
#Debug
teste_predicted.sum()

832

In [44]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89      5964
           1       0.96      0.81      0.88      6036

    accuracy                           0.88     12000
   macro avg       0.89      0.89      0.88     12000
weighted avg       0.89      0.88      0.88     12000



# **Logistic sigmoid**

In [45]:
lg=LogisticRegression(penalty='l2',class_weight='balanced',solver='sag',max_iter=5000).fit(X_train,Y_train)

In [46]:
y_perd=lg.predict(X_test)
accuracy_score(Y_test, y_perd)

0.8675833333333334

In [47]:
teste_predicted=lg.predict(features_test)

In [48]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_LSigmoid.csv', index=False)

In [49]:
teste_predicted.sum()

968

In [50]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87      5964
           1       0.87      0.86      0.87      6036

    accuracy                           0.87     12000
   macro avg       0.87      0.87      0.87     12000
weighted avg       0.87      0.87      0.87     12000



# **MLP**

In [51]:
mlp= MLPClassifier(hidden_layer_sizes=(250,40), max_iter=300,solver="adam",learning_rate_init=0.0001,verbose=True).fit(X_train,Y_train)

Iteration 1, loss = 0.64923253
Iteration 2, loss = 0.57156420
Iteration 3, loss = 0.49989829
Iteration 4, loss = 0.43806958
Iteration 5, loss = 0.38723669
Iteration 6, loss = 0.34141754
Iteration 7, loss = 0.30666829
Iteration 8, loss = 0.27665590
Iteration 9, loss = 0.24882694
Iteration 10, loss = 0.22855150
Iteration 11, loss = 0.21135745
Iteration 12, loss = 0.19222699
Iteration 13, loss = 0.17995885
Iteration 14, loss = 0.16726303
Iteration 15, loss = 0.15594231
Iteration 16, loss = 0.14580126
Iteration 17, loss = 0.13712353
Iteration 18, loss = 0.13041863
Iteration 19, loss = 0.12215156
Iteration 20, loss = 0.11441532
Iteration 21, loss = 0.10726096
Iteration 22, loss = 0.10137261
Iteration 23, loss = 0.09723298
Iteration 24, loss = 0.09091265
Iteration 25, loss = 0.08683878
Iteration 26, loss = 0.08078901
Iteration 27, loss = 0.07834100
Iteration 28, loss = 0.07559035
Iteration 29, loss = 0.06859622
Iteration 30, loss = 0.06602388
Iteration 31, loss = 0.06316893
Iteration 32, los

In [52]:
y_perd=mlp.predict(X_test)
accuracy_score(Y_test, y_perd)

0.9750833333333333

In [53]:
Y_test.sum()

6036

In [54]:
teste_predicted=mlp.predict(features_test)
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
#data.to_csv('sobel_MLP.csv', index=False)
data.to_csv('predict_MLP.csv', index=False)#'predict_MLP.csv'

In [55]:
teste_predicted.sum()

1010

In [56]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      5964
           1       0.98      0.97      0.98      6036

    accuracy                           0.98     12000
   macro avg       0.98      0.98      0.98     12000
weighted avg       0.98      0.98      0.98     12000



# **Naive Bayes**

In [57]:
nb = GaussianNB().fit(X_train,Y_train)

In [58]:
y_perd=nb.predict(X_test)
accuracy_score(Y_test, y_perd)

0.5285833333333333

In [59]:
teste_predicted=nb.predict(features_test)
teste_predicted.sum()

158

In [60]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_NBayes.csv', index=False)
#data.to_csv('sobel_NBayes.csv', index=False)

In [61]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.51      0.94      0.66      5964
           1       0.67      0.12      0.21      6036

    accuracy                           0.53     12000
   macro avg       0.59      0.53      0.44     12000
weighted avg       0.59      0.53      0.43     12000



# **SVM**

In [62]:
svm = SVC(kernel= 'rbf', gamma='scale',class_weight='balanced',verbose=True ).fit(X_train,Y_train)

[LibSVM]

In [63]:
y_perd=svm.predict(X_test)
accuracy_score(Y_test, y_perd)

0.9695

In [64]:
teste_predicted=svm.predict(features_test)
teste_predicted.sum()

1005

In [65]:
teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_predicted])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_SVM.csv', index=False)

In [66]:
print(classification_report(Y_test,y_perd))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      5964
           1       0.97      0.97      0.97      6036

    accuracy                           0.97     12000
   macro avg       0.97      0.97      0.97     12000
weighted avg       0.97      0.97      0.97     12000



# **CNN**

In [67]:
data_train = pd.read_csv('dataset_treino.csv')
data_teste = pd.read_csv('dataset_teste.csv')

Y = data_train.iloc[:,0].astype(int)
X = data_train.iloc[:,1:]
X_test = data_teste.to_numpy()
X_test = X_test/255

X_train, X_val,Y_train,Y_val = train_test_split(X,Y, test_size=0.30,shuffle=True, random_state=42) 

Y_train = Y_train.to_numpy()
X_train = X_train.to_numpy()
Y_val=Y_val.to_numpy()
X_val=X_val.to_numpy()


X_train = X_train/255
X_val=X_val/255

In [68]:
X_train=X_train.reshape(28000,28,28)
X_val=X_val.reshape(12000,28,28)
X_test=X_test.reshape(2000,28,28)

In [69]:
Y_train.shape

(28000,)

In [70]:
#definição de modelo
model = tf.keras.models.Sequential([
tf.keras.layers.Convolution2D(16, (3,3), activation='relu', input_shape=(28,28,1), padding='same'),
tf.keras.layers.BatchNormalization(),

tf.keras.layers.Convolution2D(32, (3,3), activation='relu', padding='same'),
tf.keras.layers.BatchNormalization(),         

tf.keras.layers.Convolution2D(64, (3,3), activation='relu', padding='same'),
tf.keras.layers.BatchNormalization(),    
          
tf.keras.layers.Flatten(),

tf.keras.layers.Dense(128, activation='relu'), #adding the hidden layer
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(2,activation='softmax')#softmax#2
])

In [71]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 16)       64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 64)       2

In [ ]:
#compiling the model        
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
history = model.fit(X_train,Y_train, epochs = 8, validation_split=0.2)



Epoch 1/8
700/700 [==============================] - 90s 127ms/step - loss: 0.6119 - accuracy: 0.7733 - val_loss: 0.2228 - val_accuracy: 0.9171
Epoch 2/8
700/700 [==============================] - 85s 122ms/step - loss: 0.1716 - accuracy: 0.9292 - val_loss: 0.0586 - val_accuracy: 0.9820
Epoch 3/8
700/700 [==============================] - 82s 117ms/step - loss: 0.0890 - accuracy: 0.9648 - val_loss: 0.0878 - val_accuracy: 0.9654
Epoch 4/8
700/700 [==============================] - 83s 119ms/step - loss: 0.0632 - accuracy: 0.9768 - val_loss: 0.0331 - val_accuracy: 0.9879
Epoch 5/8
700/700 [==============================] - 81s 115ms/step - loss: 0.0643 - accuracy: 0.9786 - val_loss: 0.0254 - val_accuracy: 0.9911
Epoch 6/8
700/700 [==============================] - 81s 116ms/step - loss: 0.0401 - accuracy: 0.9854 - val_loss: 0.0397 - val_accuracy: 0.9857
Epoch 7/8
700/700 [==============================] - 84s 120ms/step - loss: 0.0435 - accuracy: 0.9868 - val_loss: 0.0241 - val_accuracy:

In [ ]:
plt.plot(history.history['accuracy'], label='acc_train')
plt.plot(history.history['val_accuracy'], label='acc_test')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='loss_train')
plt.plot(history.history['val_loss'], label='loss_test')
plt.legend()
plt.show()

In [ ]:
teste_predicted=model.predict(X_test)
print(teste_predicted)
teste_predicted.shape

In [ ]:
teste_pred=np.arange(2000).reshape(2000,1)
for i in range(2000):
    if(teste_predicted[i,0] > teste_predicted[i,1]):
        teste_pred[i,0]=0
    else:
        teste_pred[i,0]=1

In [ ]:
teste_pred.sum()

In [ ]:
#teste_predicted=teste_predicted.reshape(2000,1)
d=np.arange(2000).reshape(2000,1)
#data=np.column_stack((d,teste_predicted))
data=np.hstack([d, teste_pred])
data=pd.DataFrame(data,columns=["Id","Category"])
data.to_csv('predict_CNN.csv', index=False)